## Project: Toxic Comment Classifier

Build a model that can filter user comments according to the degree of harmfulness of the language:
1. Preprocess the text by eliminating the set of tokens that do not make significant contribution at the semantic level
2. Transform the text corpus into sequences
3. Build a Deep Learning model including recurrent layers for a multilabel classification task
4. At prediction time, the model must return a vector containing a 1 or a 0 at each label in the dataset (toxic, severe_toxic, obscene, threat, insult, identity_hate). In this way, a non-harmful comment will be classified by a vector of only 0s [0,0,0,0,0]. In contrast, a dangerous comment will exhibit at least a 1 among the 6 labels.

### Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import re
import nltk
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
BASE_URL = "https://s3.eu-west-3.amazonaws.com/profession.ai/datasets/"
df = pd.read_csv(BASE_URL+"Filter_Toxic_Comments_dataset.csv")
print(f"Dataset shape: {df.shape}")
print(df.head(25))

Dataset shape: (159571, 8)
                                         comment_text  toxic  severe_toxic  \
0   Explanation\nWhy the edits made under my usern...      0             0   
1   D'aww! He matches this background colour I'm s...      0             0   
2   Hey man, I'm really not trying to edit war. It...      0             0   
3   "\nMore\nI can't make any real suggestions on ...      0             0   
4   You, sir, are my hero. Any chance you remember...      0             0   
5   "\n\nCongratulations from me as well, use the ...      0             0   
6        COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1             1   
7   Your vandalism to the Matt Shirvington article...      0             0   
8   Sorry if the word 'nonsense' was offensive to ...      0             0   
9   alignment on this subject and which are contra...      0             0   
10  "\nFair use rationale for Image:Wonju.jpg\n\nT...      0             0   
11  bbq \n\nbe a man and lets discuss

### Data pre-processing

In [ ]:
nltk.download('stopwords')

def preprocess_text(text):  #lower case conversion, Removal of special characters and stopwords
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

df['cleaned_comment'] = df['comment_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
MAX_WORDS = 10000
MAX_LENGTH = 200

tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(df['cleaned_comment'])
sequences = tokenizer.texts_to_sequences(df['cleaned_comment'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_LENGTH)

label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] # Class labels
y = df[label_columns].values

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

### Model


*   LSTM layers are effective in capturing long-term dependencies in the text, which are crucial for understanding the context and tone of the commentary. The two-level structure allows deeper processing of sequential features.

*   The fully connected layer with 64 neurons and ReLU activation introduces nonlinearity into the model.


*   The 50% dropout helps prevent overfitting, increasing the generalization of the model.

*   Final dense layer with 6 neurons (one for each class) and sigmoid activation, sigmoid is appropriate for multi-label classification, producing independent probabilities for each class.











In [ ]:
model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LENGTH),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', 'mse'])

model.build(input_shape=(None, MAX_LENGTH))
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 200, 64)             │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,344,326 (5.13 MB)

 Trainable params: 1,344,326 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/10
798/798 ━━━━━━━━━━━━━━━━━━━━ 511s 635ms/step - accuracy: 0.2191 - loss: 0.3616 - mse: 0.1112 - val_accuracy: 0.9943 - val_loss: 0.1417 - val_mse: 0.0344
Epoch 2/10
798/798 ━━━━━━━━━━━━━━━━━━━━ 550s 620ms/step - accuracy: 0.5693 - loss: 0.1604 - mse: 0.0377 - val_accuracy: 0.9835 - val_loss: 0.0926 - val_mse: 0.0254
Epoch 3/10
798/798 ━━━━━━━━━━━━━━━━━━━━ 0s 572ms/step - accuracy: 0.6164 - loss: 0.0888 - mse: 0.0227

In [ ]:
#Prediction function definition
def predict_toxicity(text):
    cleaned = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=MAX_LENGTH)
    prediction = model.predict(padded)
    binary_prediction = (prediction > 0.5).astype(int)[0]
    classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] # Lista delle classi

    positive_classes = [classes[i] for i, value in enumerate(binary_prediction) if value == 1]

    if positive_classes == []:
      print("The text sample provided is approved")
    else:
        print(f"The text sample provided is classified as: {positive_classes}")

    return binary_prediction, positive_classes

### Model and tokenizer export

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Create a directory for your project in Google Drive
project_dir = '/content/drive/My Drive/Colab Notebooks/model_parameters'
os.makedirs(project_dir, exist_ok=True)

# Save the model
model.save(os.path.join(project_dir, 'RNN_model.h5'))

# Save the tokenizer
import pickle
with open(os.path.join(project_dir, 'tokenizer.pickle'), 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save configuration
import json
config = {
    'MAX_WORDS': MAX_WORDS,
    'MAX_LENGTH': MAX_LENGTH,
    'label_columns': label_columns
}
with open(os.path.join(project_dir, 'config.json'), 'w') as f:
    json.dump(config, f)

print(f"All files saved in Google Drive: {project_dir}")

In [ ]:
sample_text = "You are an idiot"
result = predict_toxicity(sample_text)

In [ ]:
sample_text = "Good job bro, keep it up!"
result = predict_toxicity(sample_text)

In [ ]:
sample_text = "I've never seen such a bad movie!"
result = predict_toxicity(sample_text)

In [ ]:
sample_text = "The pasta was horrible, I'll never go there again"
result = predict_toxicity(sample_text)

### TRY WITH YOUR SENTENCE

In [ ]:
sentence = input("Enter a sentence: ")
result = predict_toxicity(sentence)